In [5]:
import pandas as pd
df = pd.read_csv('data/amazon_cellphones_binary.csv')
df

,asin,reviewText,overall,bin_y
0,B007D6J64K,probably favorite cover super sassy protective...,5,1
1,B007D6J64K,case_protects phone damage,5,1
2,B007D6J64K,nice,4,1
3,B007D6J64K,wa another favorite one thanks quality pricing,5,1
4,B007D6J64K,decent_case lot_of_protection,5,1
...,...,...,...,...
26840,B0096QI0QK,easy_to_put phone protects really_well obvious...,5,1
26841,B0096QI0QK,much_better quality expected price great_alter...,5,1
26842,B0096QI0QK,one_of_the_best screen_protector used applicat...,4,1
26843,B0096QI0QK,kit included microfiber_cloth soft card help a...,5,1


In [6]:
!pip install graphviz



0

In [7]:
df_not_na = df[~(df['reviewText'].isna()) & ~(df['bin_y']==-1)]
x_0 = df_not_na['reviewText']
y = df_not_na['bin_y'].tolist()

### preprocessing

In [8]:
!pip install nltk

0

In [9]:
import nltk; nltk.download('stopwords'); nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stop = stopwords.words('english')
stop.extend(['good', 'bad', 'dont'])
lemmatizer = WordNetLemmatizer()

x = x_0.str.lower().str.replace('[^\w\s]','')
x = x.str.split()
x = x.apply(lambda x: [lemmatizer.lemmatize(item) for item in x])
x = x.apply(lambda x: [item for item in x if item not in stop])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/srv/conda/envs/notebook/bin/xpython:8: FutureWarning: The default value of regex will change from True to False in a future version.


### import embedding models

In [10]:
!pip install gensim

0

In [11]:
from gensim.models import KeyedVectors 
embeddings = KeyedVectors.load_word2vec_format('data/ft_model.bin', binary=True)
# embeddings = KeyedVectors.load_word2vec_format('models/ft_vectors_cbow5005_0.05.vec', binary=False, unicode_errors='ignore')

### text to embeddings

#### tokenize and pad

In [12]:
!pip install keras

0

In [13]:
!pip install tensorflow

0

In [14]:
!pip install --upgrade tensorflow

0

In [15]:
import keras
import sklearn

In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

max_length = 20

t = Tokenizer()
t.fit_on_texts(x)
vocab_size = len(t.word_index) + 1
emb_size = embeddings.vector_size

encoded_docs = t.texts_to_sequences(x)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs[0])

x_train, x_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.3, random_state=42, stratify=y)

[  201   637    16   371  9910   301 11961     1     2  8202   148    25
   461   270   105  4033  1075     9    31     0]


#### create embedding weights matrix

In [17]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(1)

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, emb_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if word in embeddings.vocab:
            weight_matrix[i] = embedding[word]
    return weight_matrix

### create embedding features

### LSTM classification

In [34]:
!pip install python-magic

0

In [25]:
!pip install --upgrade ipython

     |████████████████████████████████| 784 kB 4.6 MB/s eta 0:00:01     |█▎                              | 30 kB 14.9 MB/s eta 0:00:01
  Attempting uninstall: ipython
    Found existing installation: ipython 7.20.0
    Uninstalling ipython-7.20.0:
      Successfully uninstalled ipython-7.20.0


0

In [36]:
import magics

ModuleNotFoundError: No module named 'magics'

In [35]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, LSTM, Bidirectional, GRU, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# get vectors in the right order
embedding_vectors = get_weight_matrix(embeddings, t.word_index)
print(embedding_vectors.shape)
e = Embedding(vocab_size, emb_size, weights=[embedding_vectors], input_length=max_length, trainable=False)

#params
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
n_labels = 2

# create the model
def baseline_model(optimizer=opt):
    model = Sequential()
    model.add(e)
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(n_labels, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    return model

model = KerasClassifier(build_fn=baseline_model)
# batch_size = [32, 64, 128]
# epochs = [5, 10, 20]
# optimizer = ['rmsprop', 'Adam']
batch_size = [64]
epochs = [5]
optimizer = ['Adam']
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1_weighted', verbose=2)
print('***CROSS VALIDATED GRID SEARCH***')
%time grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

(21664, 100)
***CROSS VALIDATED GRID SEARCH***


ValueError: magics not found

### predict test set

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
#results on the test set for the best model
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('***RESULTS ON TEST SET***')
print("accuracy_score", accuracy)
print("f1_score", f1)
print('\n')
print(classification_report(y_test, y_pred))

### Save Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
# model = best_model.fit(x_train, y_train)
best_model.model.save('lstm_models/best_lstm.h5')

<img src='img/grid_LSTM_results.PNG'>

<img src='img/48workers.PNG'>